In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


spark = SparkSession.builder.appName("UberDataAnalysis").getOrCreate()
df = spark.read.csv("dataset.csv", header=True, inferSchema=True) 

In [7]:
#Renaming the columns for usability 

column_mapping = {
    "Time (Local)": "Local_Time",
    "Eyeballs ": "Eyeballs",
    "Zeroes ": "Zeroes",
    "Completed Trips ": "Completed_Trips",
    "Requests ": "Requests",
    "Unique Drivers": "Unique_Drivers"
}

for old_col, new_col in column_mapping.items():
    df = df.withColumnRenamed(old_col, new_col)
df.show()

+---------+----------+--------+------+---------------+--------+--------------+
|     Date|Local_Time|Eyeballs|Zeroes|Completed_Trips|Requests|Unique_Drivers|
+---------+----------+--------+------+---------------+--------+--------------+
|10-Sep-12|         7|       5|     0|              2|       2|             9|
|10-Sep-12|         8|       6|     0|              2|       2|            14|
|10-Sep-12|         9|       8|     3|              0|       0|            14|
|10-Sep-12|        10|       9|     2|              0|       1|            14|
|10-Sep-12|        11|      11|     1|              4|       4|            11|
|10-Sep-12|        12|      12|     0|              2|       2|            11|
|10-Sep-12|        13|       9|     1|              0|       0|             9|
|10-Sep-12|        14|      12|     1|              0|       0|             9|
|10-Sep-12|        15|      11|     2|              1|       2|             7|
|10-Sep-12|        16|      11|     2|              

In [18]:
#Creating new 'timestamp' column

df = df.withColumn("Timestamp", from_unixtime(unix_timestamp(col("Date"), "yyyy-MM-dd") + col("Local_Time") * 3600))
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Local_Time: integer (nullable = true)
 |-- Eyeballs: integer (nullable = true)
 |-- Zeroes: integer (nullable = true)
 |-- Completed_Trips: integer (nullable = true)
 |-- Requests: integer (nullable = true)
 |-- Unique_Drivers: integer (nullable = true)
 |-- Timestamp: string (nullable = true)



In [8]:
# Which date had the most completed trips during the two-week period?
from pyspark.sql.functions import max
completed_trips_by_date = df.groupBy("Date").sum("Completed_Trips")

date_with_most_completed_trips = completed_trips_by_date \
    .orderBy("sum(Completed_Trips)", ascending=False) \
    .select("Date") \
    .first()["Date"]

print(date_with_most_completed_trips)

22-Sep-12


In [19]:
df.show()

+----------+----------+--------+------+---------------+--------+--------------+-------------------+
|      Date|Local_Time|Eyeballs|Zeroes|Completed_Trips|Requests|Unique_Drivers|          Timestamp|
+----------+----------+--------+------+---------------+--------+--------------+-------------------+
|2012-09-10|         7|       5|     0|              2|       2|             9|2012-09-10 07:00:00|
|2012-09-10|         8|       6|     0|              2|       2|            14|2012-09-10 08:00:00|
|2012-09-10|         9|       8|     3|              0|       0|            14|2012-09-10 09:00:00|
|2012-09-10|        10|       9|     2|              0|       1|            14|2012-09-10 10:00:00|
|2012-09-10|        11|      11|     1|              4|       4|            11|2012-09-10 11:00:00|
|2012-09-10|        12|      12|     0|              2|       2|            11|2012-09-10 12:00:00|
|2012-09-10|        13|       9|     1|              0|       0|             9|2012-09-10 13:00:00|


In [22]:
#What was the highest number of completed trips within a 24-hour period?

df = df.withColumn("Timestamp", col("Timestamp").cast("timestamp"))

# Group the data by 24-hour windows and sum the completed trips
completed_trips_by_window = df \
    .groupBy(window("Timestamp", "24 hours")) \
    .agg(sum("Completed_Trips").alias("Total_Completed_Trips")) \
    .orderBy("Total_Completed_Trips", ascending=False)

# Get the highest number of completed trips within a 24-hour period
highest_completed_trips_in_24_hours = completed_trips_by_window \
    .select("Total_Completed_Trips") \
    .first()["Total_Completed_Trips"]

print(f"Highest number of completed trips within a 24-hour period: {highest_completed_trips_in_24_hours}") 

Highest number of completed trips within a 24-hour period: 257


In [31]:
#Which hour of the day had the most requests during the two-week period?

from pyspark.sql.functions import hour, sum

hourly_requests = df \
    .groupBy(hour("Timestamp").alias("hour")) \
    .agg(sum("Requests").alias("total_requests")) \
    .orderBy("total_requests", ascending=False)

most_requested_hour = hourly_requests.select("hour").first()[0]
print("The hour with the most requests is:", most_requested_hour)



The hour with the most requests is: 23


In [32]:
#What percentages of all zeroes during the two-week period occurred on weekends (Friday at 5 pm to Sunday at 3 am)?

from pyspark.sql.functions import dayofweek, hour

weekend_zeros = df.filter((hour("Timestamp") >= 17) | (hour("Timestamp") < 3)).filter((dayofweek("Date") == 6) | (dayofweek("Date") == 7)).agg(sum("Zeroes").alias("weekend_zeros")).collect()[0]["weekend_zeros"]

total_zeros = df.agg(sum("Zeroes").alias("total_zeros")).collect()[0]["total_zeros"]

percent_weekend_zeros = weekend_zeros / total_zeros * 100

print("The percentage of zeros that occurred on weekends is:", percent_weekend_zeros, "%")

The percentage of zeros that occurred on weekends is: 29.111266620014 %


In [37]:
#What is the weighted average ratio of completed trips per driver during the two-week period?

from pyspark.sql.functions import avg

weighted_avg = df.withColumn("completed_per_driver", df["Completed_Trips"] / df["Unique_Drivers"]) \
                 .groupBy("Date", "Local_Time") \
                 .agg(avg("completed_per_driver").alias("avg_completed_per_driver"), sum("Completed_Trips").alias("total_completed_trips")) \
                 .withColumn("weighted_ratio", col("avg_completed_per_driver") * col("total_completed_trips")) \
                 .agg(sum("weighted_ratio") / sum("total_completed_trips")).collect()[0][0]

print("The weighted average ratio of completed trips per driver is:", weighted_avg)


The weighted average ratio of completed trips per driver is: 0.8276707747535552


### The consecutive_sum column represents the sum of unique requests over an 8-hour consecutive window, including the current hour. In this case, the sum is 80, indicating the total number of unique requests observed over the 8 consecutive hours, where 8:00 PM is part of that window.
### Businesses can use this insight for operational planning, resource allocation, or targeted promotions during this peak demand period.

In [42]:
#In drafting a driver schedule in terms of 8 hours shifts, when are the busiest 8 consecutive hours over the two-week period in terms of unique requests? A new shift starts every 8 hours. Assume that a driver will work the same shift each day.


from pyspark.sql.functions import col, hour, countDistinct
from pyspark.sql.window import Window

hourly_unique_requests = (df
  .groupBy(hour("Timestamp").alias("hour"))
  .agg(countDistinct("Requests").alias("unique_requests"))
)

window = Window.orderBy(col("unique_requests").desc()).rowsBetween(0, 7)
busiest_8_consecutive_hours = (hourly_unique_requests
  .select("*", sum("unique_requests").over(window).alias("consecutive_sum"))
  .orderBy(col("consecutive_sum").desc()) 
  .limit(1)
)

busiest_8_consecutive_hours.show()

24/01/15 14:16:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/15 14:16:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/15 14:16:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/15 14:16:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/15 14:16:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----+---------------+---------------+
|hour|unique_requests|consecutive_sum|
+----+---------------+---------------+
|  20|             12|             80|
+----+---------------+---------------+



24/01/15 14:16:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/15 14:16:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/15 14:16:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/15 14:16:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


### The recommendation is to add 5 drivers during the 2:00 AM hour. This suggestion is based on the analysis of the average number of requests per unique driver during that specific hour.

In [54]:
#If you could add 5 drivers to any single hour of every day during the two-week period, which hour should you add them to? Hint: Consider both rider eyeballs and driver supply when choosing.

from pyspark.sql.functions import sum, countDistinct, desc

requests_per_driver = (df.groupBy('Local_Time').agg(
    (sum('Requests') / countDistinct('Unique_Drivers')).alias('requests_per_driver'))
)

requests_per_driver.orderBy(desc('requests_per_driver')).show(1)

+----------+-------------------+
|Local_Time|requests_per_driver|
+----------+-------------------+
|         2|               20.0|
+----------+-------------------+
only showing top 1 row



In [58]:
#Looking at the data from all two weeks, which time might make the most sense to consider a true "end day" instead of midnight? (i.e when are supply and demand at both their natural minimums)

avg_trips_and_drivers = (df.groupBy('Local_Time').agg(avg('Completed_Trips').alias('avg_completed_trips'),avg('Unique_Drivers').alias('avg_unique_drivers')
))

avg_trips_and_drivers.orderBy('avg_completed_trips', 'avg_unique_drivers').show(1)

+----------+-------------------+------------------+
|Local_Time|avg_completed_trips|avg_unique_drivers|
+----------+-------------------+------------------+
|         4|0.14285714285714285|0.6428571428571429|
+----------+-------------------+------------------+
only showing top 1 row

